In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from librosa import core, onset, feature, display
import soundfile as sf
import umap
from IPython.display import Audio
import sklearn

In [3]:
df = pd.read_csv("./songs/predict/predict.csv")
df.head()

,file_id,genus,species
0,1,Unknown,species


In [12]:
def load_audio(file_id):
    data, samplerate = sf.read("./songs/predict/xc"+str(file_id)+".flac")
    s = len(data)/samplerate
    sg = feature.melspectrogram(y=data, sr=samplerate, hop_length=512)
    
    # Take mean amplitude M from frame with highest energy
    centerpoint = np.argmax(sg.mean(axis=0))
    M = sg[:,centerpoint].mean()
    
    # Filter out all frames with energy less than 5% of M
    mask = sg.mean(axis=0)>=M/20

    audio_mask = np.zeros(len(data), dtype=bool)
    for i in range(0,len(mask)):
        audio_mask[i*512:] = mask[i]
    return sg, mask, data, audio_mask, samplerate

In [13]:
df['length'] = np.zeros(len(df))

waves = {}


for file_id in df['file_id']:
    sg, mask, data, audio_mask, sample_rate = load_audio(file_id)
    waves[file_id] = data[audio_mask]
    df.loc[df['file_id'] == file_id,'length'] = len(data[audio_mask])

In [14]:
df['windows'] = df['length'].apply(lambda x: int(x/6.144000e+03))
df

,file_id,genus,species,length,windows
0,1,Unknown,species,116736.0,19


In [15]:
n_windows = df.groupby('species')['windows'].sum().min()
n_windows

np.int64(19)

In [16]:
windows = {}

for file_id in df['file_id']:
    wave = waves[file_id]
    species = df[df['file_id']==file_id]['genus'].values[0] + "_" + df[df['file_id']==file_id]['species'].values[0]
    if species not in windows:
        windows[species] = []
    for i in range(0, int(len(wave)/6.144000e+03)):
        windows[species].append(wave[i:int(i+6.144000e+03)])

In [17]:
windows_fixed = {}

for species in windows.keys():
    windows_fixed[species] = []
    ws = windows[species]
    index = np.random.choice(len(ws), n_windows, replace=False)
    for i in range(0, len(ws)):
        if i in index:
            windows_fixed[species].append(ws[i])

In [21]:
new_dataset = pd.DataFrame()

for species in windows_fixed.keys():
    for i in range(0,n_windows):
        data_point = {'species':species.split('_')[1], 'genus':species.split('_')[0]}
        spec_centroid = feature.spectral_centroid(y=windows_fixed[species][i])[0]
        chroma = feature.chroma_stft(y=windows_fixed[species][i], sr=sample_rate)
        for j in range(0,13):
            data_point['spec_centr_'+str(j)] = spec_centroid[j]
            for k in range(0,12):
                data_point['chromogram_'+str(k)+"_"+str(j)] = chroma[k,j]
        data_point = pd.DataFrame([data_point])
        new_dataset = pd.concat([new_dataset, data_point],ignore_index=True)

new_dataset.head()

c:\Users\K Samarth N Kamath\Bird-Vocalization-Detection-System\.venv\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


,species,genus,spec_centr_0,chromogram_0_0,chromogram_1_0,chromogram_2_0,chromogram_3_0,chromogram_4_0,chromogram_5_0,chromogram_6_0,...,chromogram_2_12,chromogram_3_12,chromogram_4_12,chromogram_5_12,chromogram_6_12,chromogram_7_12,chromogram_8_12,chromogram_9_12,chromogram_10_12,chromogram_11_12
0,species,Unknown,3181.637582,0.000833,0.000790,0.000987,0.001147,0.001561,0.013176,0.440830,...,0.002372,0.003946,0.011741,0.247158,1.0,0.207101,0.008497,0.002119,0.001486,0.001415
1,species,Unknown,3364.629079,0.000767,0.000701,0.000860,0.000992,0.001378,0.012949,0.441586,...,0.002030,0.003495,0.011064,0.246104,1.0,0.207251,0.008606,0.002090,0.001376,0.001245
2,species,Unknown,3611.369476,0.000640,0.000555,0.000634,0.000688,0.001099,0.012565,0.443388,...,0.001483,0.002760,0.009939,0.244820,1.0,0.206709,0.008615,0.001952,0.001175,0.000951
3,species,Unknown,3358.804555,0.000764,0.000771,0.000896,0.000948,0.001534,0.013229,0.446249,...,0.001901,0.003295,0.010764,0.247900,1.0,0.204385,0.008295,0.001881,0.001278,0.001108
4,species,Unknown,3194.050269,0.000821,0.000860,0.001014,0.001076,0.001704,0.013525,0.447527,...,0.002392,0.003939,0.011764,0.250598,1.0,0.203025,0.008121,0.001904,0.001424,0.001322


In [22]:
features= list(new_dataset.columns)
features.remove('species')
features.remove('genus')

X = new_dataset[features].values

In [24]:
loaded_model = pickle.load(open('svm.sav', 'rb'))

In [25]:
ans=loaded_model.predict(X)
dic={}
s=set(ans)
for i in s:
    dic[i]=sum([1 for j in list(ans) if i==j])
dic['jadf']=12
sol=None
res=None
for i in dic.items():
    if sol is None or sol<i[1]:
        sol=i[1]
        res=i[0]
res

'familiaris'

In [26]:
loaded_model = pickle.load(open('naive.sav', 'rb'))

In [27]:
ans=loaded_model.predict(X)
dic={}
s=set(ans)
for i in s:
    dic[i]=sum([1 for j in list(ans) if i==j])
dic['jadf']=12
sol=None
res=None
for i in dic.items():
    if sol is None or sol<i[1]:
        sol=i[1]
        res=i[0]
res

np.str_('palustris')